In [1]:
import scipy
from scipy import array
import time
tStart = time.time()

def quad_routine(func, a, b, x_list, w_list):
    c_1 = (b-a)/2.0
    c_2 = (b+a)/2.0
    eval_points = map(lambda x: c_1*x+c_2, x_list)
    func_evals = list(map(func, eval_points))    # Python 3: make a list here
    return c_1 * sum(array(func_evals) * array(w_list))

def quad_gauss_7(func, a, b):
    x_gauss = [-0.949107912342759, -0.741531185599394, -0.405845151377397, 0, 0.405845151377397, 0.741531185599394, 0.949107912342759]
    w_gauss = array([0.129484966168870, 0.279705391489277, 0.381830050505119, 0.417959183673469, 0.381830050505119, 0.279705391489277,0.129484966168870])
    return quad_routine(func,a,b,x_gauss, w_gauss)

def quad_kronrod_15(func, a, b):
    x_kr = [-0.991455371120813,-0.949107912342759, -0.864864423359769, -0.741531185599394, -0.586087235467691,-0.405845151377397, -0.207784955007898, 0.0, 0.207784955007898,0.405845151377397, 0.586087235467691, 0.741531185599394, 0.864864423359769, 0.949107912342759, 0.991455371120813]
    w_kr = [0.022935322010529, 0.063092092629979, 0.104790010322250, 0.140653259715525, 0.169004726639267, 0.190350578064785, 0.204432940075298, 0.209482141084728, 0.204432940075298, 0.190350578064785, 0.169004726639267, 0.140653259715525,  0.104790010322250, 0.063092092629979, 0.022935322010529]
    return quad_routine(func,a,b,x_kr, w_kr)

class Memoize:                     
    def __init__(self, func):
        self.func = func
        self.eval_points = {}
    def __call__(self, *args):
        if args not in self.eval_points:
            self.eval_points[args] = self.func(*args)
        return self.eval_points[args]

def quad(func,a,b):
    ''' Output is the 15 point estimate; and the estimated error '''
    func = Memoize(func) 
    g7 = quad_gauss_7(func,a,b)
    k15 = quad_kronrod_15(func,a,b)
    return (k15)

def error(func,a,b):
    func = Memoize(func) 
    g7 = quad_gauss_7(func,a,b)
    k15 = quad_kronrod_15(func,a,b)    
    return ((200*scipy.absolute(g7-k15))**1.5)

In [2]:
import matplotlib.pyplot as plt
import numpy as np

In [3]:
def f(x):
    return (1/x)*np.cos((1/x)*np.log(x))

def df(x):
    return ((np.log(x)-1)*(np.sin(np.log(x)/x))-x*np.cos(np.log(x)/x))/(x*x*x)
z=1000000

In [4]:
def g(k):
    def f(x):
        return (1/x)*np.log(x)+(k)*np.pi
    return f
def dg(x):
    return (1-np.log(x))/(x*x)

In [5]:
def newton(
    func,
    d_func,
    x_0,
    tolerance,
    max_iterations,
    report_history
):
    x_n = x_0
    num_iterations = 0
    
    while True:
        
        f_of_x_n = func(x_n)
        
        error = abs(f_of_x_n)
        
        if error < tolerance:
            #print('Found solution after',num_iterations,'iterations.')
            return x_n
        
        d_f_of_x_n = d_func(x_n)
        
        if d_f_of_x_n == 0:
            #print('Zero derivative. No solution found.')
            return None
        
        if num_iterations < max_iterations:
            num_iterations += 1
            
            x_n = x_n - f_of_x_n / d_f_of_x_n
            
        else:
            #print('Terminate since reached the maximum iterations.')
            #print(error)
            return x_n


In [6]:
a=[]
root=1
for i in range(1,z,1):
    root = newton(g(i), dg, root*0.5, max_iterations=100, tolerance=1e-15, report_history=False)
    a.append(root)

In [7]:
M = quad(lambda x: (1/x)*scipy.exp(1j*(scipy.log(x)/x)), a[0],1)
for i in range(0,z-5,1):
    M += (quad(lambda x: (1/x)*scipy.exp(1j*(scipy.log(x)/x)), a[i+1],a[i]))
print(M)
tEnd = time.time()

print (tEnd - tStart)

(0.32336743174640686-0.45936061810895573j)
470.3708837032318


In [8]:
M-0.32336743167777876139

(6.862810320029666e-11-0.45936061810895573j)

In [9]:
##err = 0
##for i in range(1,z-5,1):
 ##   err += (error(lambda x: (1/x)*scipy.exp(1j*(scipy.log(x)/x)), a[i+1],a[i]))
##print(err)